<a href="https://colab.research.google.com/github/jcmachicaocuf/CUF__NLP_LLM/blob/main/nlp__gdmk_extractor_periodicos_w.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracción de vocabularios de periódicos locales

In [ ]:
#@title Librerias
import urllib3
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import nltk

In [ ]:
#@title Carga de Webs Fuente
web_pages = [
              'https://gestion.pe/',
              'https://larepublica.pe',
              'https://www.expreso.com.pe/politica/',
              'http://www.laindustria.pe/actualidad',
              'https://diariocorreo.pe/edicion/la-libertad/?ref=menu_edi',
              'https://rpp.pe/peru/ayacucho/',
              'http://www.noticiasser.pe/cajamarca/'
              'http://www.servindi.org/peru',
              'https://radioamericana.com.pe/category/moquegua/',
              'http://www.radiouno.pe/noticias/locales/',
              'https://www.dipromin.com/category/noticias/mineria/',
              'https://www.radiocutivalu.org/secciones/piura/',
              'https://elcomercio.pe/politica/',
              'https://www.actualidadambiental.pe/',
              'https://elperuano.pe/seccion.aspx?sec=1',
              'https://elcholo.com.pe/category/social/',
              'https://canaln.pe/alerta-noticias',
              'http://www.radioyaravi.org.pe/nota/3',
              'https://www.pachamamaradio.org/regional',
              'http://www.radiouno.pe/noticias/locales/'

            ]

In [ ]:
#@title Explorador
noticias = []
wbs = 'http://www.laindustria.pe/actualidad'
pagina = requests.get(wbs)
soup = BeautifulSoup(pagina.content, 'html.parser')

params = [
          ['a', 'colorTextBlanco RobotoSlabBold textoTruncado'],
          ['a', 'RobotoSlabRegular colorTextoAzul'],
          ]

for tx_a in soup.findAll('a', {'class': 'colorTextBlanco RobotoSlabBold textoTruncado'}):
  noticias.append(tx_a.text)

for tx_a in soup.findAll('a', {'class': 'RobotoSlabRegular colorTextoAzul'}):
  noticias.append(tx_a.text)

noticias

['Pronabec ofrece 150 becas para escolares mujeres interesadas en carreras de ciencias',
 'Sentenciados por corrupción podrán postular este 2021 tras mecanismo de estas cuatro bancadas',
 'Los Cuellos Blancos: Fiscal de la Nación denuncia constitucionalmente a Orlando Velásquez',
 '[PODCAST] Convivir con el miedo, por Edgar Vásquez',
 'Publican decreto que otorga subsidio económico al transporte',
 'Plantean adelantar pago de gratificaciones en sector público',
 'Desde el 15 de julio se reinicia el transporte aéreo y terrestre interprovincial de pasajeros',
 'Comisión Agraria aprueba dictamen que crea fondo de salvataje para la agricultura familiar',
 'Perú sobrepasa los 10 mil fallecidos por coronavirus']

In [ ]:
#@title Extracción
noticias = []
for wbs in web_pages:
  print('***', wbs)
  pagina = requests.get(wbs)
  soup = BeautifulSoup(pagina.content, 'html.parser')
  for k in soup.findAll('div', {'class': 'title'}):
    for j in k.find_all('a', href=True):
      txa = j.text.replace('\n', '').replace('\t', '')
      if len(txa)>10:
        noticias.append([wbs, txa])
  for k in soup.findAll('article', {'class': 'LastNewsList'}):
    for j in k.find_all('a', href=True):
      txa = j.text.replace('\n', '').replace('\t', '')
      if len(txa)>10:
        noticias.append([wbs, txa])
  for k in soup.findAll('h1'):
    for j in k.find_all('a', href=True):
      txa = j.text.replace('\n', '').replace('\t', '')
      if len(txa)>10:
        noticias.append([wbs, txa])
  for k in soup.findAll('h2'):
    for j in k.find_all('a', href=True):
      txa = j.text.replace('\n', '').replace('\t', '')
      if len(txa)>10:
        noticias.append([wbs, txa])
noticias_df = pd.DataFrame(noticias)

*** https://gestion.pe/
*** https://larepublica.pe
*** https://www.expreso.com.pe/politica/
*** http://www.laindustria.pe/actualidad
*** https://diariocorreo.pe/edicion/la-libertad/?ref=menu_edi
*** https://rpp.pe/peru/ayacucho/
*** http://www.noticiasser.pe/cajamarca/http://www.servindi.org/peru
*** https://radioamericana.com.pe/category/moquegua/
*** http://www.radiouno.pe/noticias/locales/
*** https://www.dipromin.com/category/noticias/mineria/
*** https://www.radiocutivalu.org/secciones/piura/
*** https://elcomercio.pe/politica/
*** https://www.actualidadambiental.pe/
*** https://elperuano.pe/seccion.aspx?sec=1
*** https://elcholo.com.pe/category/social/
*** https://canaln.pe/alerta-noticias
*** http://www.radioyaravi.org.pe/nota/3
*** https://www.pachamamaradio.org/regional
*** http://www.radiouno.pe/noticias/locales/


In [ ]:
noticias_df.columns = ['fuente', 'titular']
noticias_df.head()

,fuente,titular
0,https://gestion.pe/,Cambios en feriados por Fiestas Patrias: ¿cómo...
1,https://gestion.pe/,Inmunidad parlamentaria: Ledesma ve difícil un...
2,https://gestion.pe/,Fiscal Domingo Pérez: esta semana es decisiva ...
3,https://gestion.pe/,Osinergmin prohíbe condicionar pagos de luz y ...
4,https://gestion.pe/,ATU: Estamos próximos a dar la viabilidad de l...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
noticias_df.to_excel('drive/My Drive/Propuestas/propuestas_geodinamica/data/results/noticias_titulares.xlsx')

In [ ]:
corpus = '. '.join(noticias_df.titular)
corpus

'Cambios en feriados por Fiestas Patrias: ¿cómo se calculará el pago de trabajadores?. Inmunidad parlamentaria: Ledesma ve difícil un referéndum ante riesgo por COVID-19. Fiscal Domingo Pérez: esta semana es decisiva para Pedro Chávarry y Tomás Gálvez. Osinergmin prohíbe condicionar pagos de luz y gas para atender reclamos por facturación. ATU: Estamos próximos a dar la viabilidad de las Líneas 3 y 4 del Metro de Lima  . Sector público: lo que más ganaron los trabajadores por laudos arbitrales . Donald Trump celebra el 4 de julio con gran evento pese al COVID-19. OMS detiene ensayos con hidroxicloroquina para tratar pacientes hospitalizados por COVID-19. 5G: ¿Cómo va su evolución y qué retos enfrentará en el Perú?. Neo Consulting: ¿Dónde compraron los peruanos durante la cuarentena?. FIDA: agricultura familiar en Perú y el camino a la ‘resiliencia’ ante cualquier crisis. Galería Indigo: El negocio online no logra igualar la experiencia directa al comprar arte . Marsh: asegurándonos un 

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.probability import FreqDist

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#@title Generación de vocabulario más frecuente
corpus_tk = word_tokenize(corpus.lower())
corpus_tk = [w for w in corpus_tk if len(w)>4]
fdist = FreqDist(corpus_tk)
fdist_df = pd.DataFrame.from_dict(fdist, orient='index').sort_values(by=[0], ascending=False)
print(len(fdist_df))
fdist_df[0].head(10)

977


ayacucho         47
covid-19         39
congreso         17
libertad         14
contra           11
personas          9
inmunidad         9
parlamentaria     9
trujillo          8
morales           8
Name: 0, dtype: int64

## Comentarios

Los pasos siguientes a este punto serían:
* Mayor validación de vocabularios
* Comparación de vocabularios por medio
* Si se hace la lectura semanal se puede hacer seguimientos por día y comparaciones

Consultas:

¿Cómo sería el proceso en el caso de conflictos de Defensoría?:

1. Se arma el vocabulario: a partir de los PDF de defensoría
2. Se envía este vocabulario al cliente para validación
3. Se hace match de vocabulario de defensoría y noticias web?
4. Se obtiene y valida primer filtrado de noticias web de interés
5. Se identifica features (regiones, tipos de conflictos pre-establecidos, actores)
6. Se agrupa tipos de noticias según features? o k-means? (a veces k-means falla mucho...)
7. Desarrollo de esquema de reporte que el cliente requiere automatizar

Procedimiento más desde el cliente:

1. Se hace un listado de medios que DEF quiere cubrir (sobre todo que cubran todos los departamentos)
2. Se recopilan criterios de calificación del conflicto. Internamente podríamos emplear varias técnicas para conectar el *criterio conceptual* con el *algoritmo*
3. Se hace una clasificación experimental usando los criterios. Parece que su interés es ahora distribución geográfica del conflicto
4. Se valida el resultado

# Extacción de Reportes de Defensoría

In [ ]:
!pip install PyPDF2

In [ ]:
#@title Librerías adicionales
import PyPDF2
import pandas as pd
import re
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ruta = 'drive/My Drive/2020 Proyectos/2020 Iniciativas con Claudia Avila/PY Monitoreo de Medios Demo/documentos fuente/'
archivos = os.listdir(ruta)
archivos

['Monitoreo de Medios N° 2792_13-06-2020.pdf',
 'Monitoreo de Medios N° 2797_19-06-2020.pdf',
 'Monitoreo de Medios N° 2785_05-06-2020.pdf',
 'Monitoreo de Medios N° 2760_07-05-2020.pdf',
 'Monitoreo de Medios N° 2747_22-04-2020.pdf',
 'Monitoreo de Medios N° 2739_14-04-2020.pdf',
 'Monitoreo de Medios N° 2733_04-04-2020.pdf',
 'Monitoreo de Medios N° 2731_02-04-2020.pdf',
 'Monitoreo de Medios N° 2794_16-06-2020.pdf',
 'Monitoreo de Medios N° 2802_25-06-2020.pdf']

In [ ]:
arch_x = archivos[2]

In [ ]:
#@title Extracción de texto PDF
tex_acum = ''
fileObj = open(ruta + arch_x, 'rb')
pdfRead = PyPDF2.PdfFileReader(fileObj)
pag = pdfRead.numPages

for i in range(0, pag):
    page = pdfRead.getPage(i)
    lector = page.extractText()
    texto = str(lector)
    tex_acum += ' ' + texto.replace('\n', '. ')
print(tex_acum)

  . * Este documento es de circulación interna y constituye un insumo periodístico para el monitoreo, análisis e . intervención en conflictos sociales..  . * La APCSG reinicia el monitoreo de medios luego de su suspensión en diciembre de 2016 . (Boletín N.º 2268).  .  . * El contenido de este documento no expresa ni insinúa ningún tipo de opinión ni posición de la Defensoría del Pueblo..  .  . http://www.defensoria.gob.pe/temas.php?des=3.  .  .  . ADJUNTÍA PARA LA PREVENCIÓN DE CONFLICTOS SOCIALES . Y LA G. OBERNABILIDAD.  .  . MONITOREO DE MEDIOS Nº 2785.  . Viernes, . 5 de junio de 2020.  .  .  . .  . Apurímac.  .  . Minera Las Bambas entregó pruebas rápidas a la provincia de Cotabambas.. -.  . La Minera . Las Bambas.  . entregó 2 mil 500 pruebas rápidas al Comando Covid. -. 19 de Cotabambas, . conformado por los alcaldes distritales, alcalde provincial, subprefectura, así como por . autoridades y representantes de la sociedad civil y comunal de dicha jurisdicción. La entrega . for. 

In [ ]:
#@title Generación de vocabulario más frecuente
corpus_tk = word_tokenize(tex_acum.lower())
corpus_tk = [w for w in corpus_tk if len(w)>4]
fdist = FreqDist(corpus_tk)
fdist_df = pd.DataFrame.from_dict(fdist, orient='index').sort_values(by=[0], ascending=False)
print(len(fdist_df))
fdist_df[0].head(10)

1124


covid           22
salud           22
nacional        21
monitoreo       19
decreto         18
conflictos      18
documento       18
defensoría      12
emergencia      12
trabajadores    12
Name: 0, dtype: int64

# Identificación de hrefs por artículo

# Experimento Extracción de Artículo Completo

In [ ]:
!pip install newspaper3k

In [ ]:
from newspaper import Article
import string
import nltk
nltk.download('punkt')
import requests
from bs4 import BeautifulSoup

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
urlx = 'https://aprimerahora.pe/2020/06/26/observan-y-piden-explicaciones-al-cambio-de-razon-social-y-el-logo-de-la-emufec/'

In [ ]:
#@title Extraer artículo de URL
article = Article(url=urlx)
article.download() #Download the article
article.parse() #Parse the article
article.nlp() #Apply Natural Language Processing (NLP)
corpus = article.text #Store the article text into corpus
corpus

'Observan y piden explicaciones al cambio de razón social y el logo de la EMUFEC\n\nRegidores de la municipalidad provincial del Cusco, hacen serias observaciones y piden explicaciones a los funcionarios de la Empresa Municipal de Festejos del Cusco EMUFEC, al cambio de la razón social y el logo de la empresa, el cual pusieron de manifiesto en las fiestas jubilares.\n\nIndicaron que estos cambios que realizaron en las recientes fiestas del mes jubilar de nuestra ciudad, fueron impuestas el cual se realizó sin el acuerdo ni análisis que debió de realizarse en la junta general de accionistas\n\n“En ningún momento hemos aprobado el cambio de la razón social de la empresa, menos el logo que en estas fiestas estrenaron los de la EMUFEC, por ello vamos a proponer en sesión de consejo para que el directorio se presente al pleno del consejo”, señalo Ricardo Almanza.\n\nEl regidor de la municipalidad provincial, señalo que nadie puede hacer lo que quiere en la empresa municipal de festejos, sin

## Método de Extracción de Contenido de Conflictos de Artículos Periódicos

1. En PDFs de Monitoreo Medios : Corpus_Monitoreo_Medios
2. Extraer vocabulario de Corpus_Monitoreo_Medios
  - Frecuencia de palabras, Filtradas por Expertos
  - Sentimientos
  - Entidades
  - Verbos


In [ ]:
#@title Explorador
noticias = []
wbs = 'https://aprimerahora.pe/'
pagina = requests.get(wbs)
soup = BeautifulSoup(pagina.content, 'html.parser')
#soup
pars = [['a', 'post-url post-title']]

seleccion = soup.findAll(pars[0][0], attrs={'class': pars[0][1]})
for sel in seleccion:
  noticias.append(sel['href'])
noticias[0:5]

['https://aprimerahora.pe/2020/06/26/autoridades-anuncian-que-hoy-se-firmando-el-contrato-para-el-hospital-antonio-lorena/',
 'https://aprimerahora.pe/2020/06/26/40-de-alcaldes-de-la-region-no-tienen-el-plan-de-contingencia-para-el-invierno/',
 'https://aprimerahora.pe/2020/06/26/se-incendia-el-balneario-turistico-de-cocalmayo-santa-teresa-la-convencion/',
 'https://aprimerahora.pe/2020/06/26/en-cusco-mas-de-9-mil-hogares-pobres-no-fueron-incluidos-en-los-bonos-economicos/',
 'https://aprimerahora.pe/2020/06/26/observan-y-piden-explicaciones-al-cambio-de-razon-social-y-el-logo-de-la-emufec/']

In [ ]:
seleccion[0]['href']

'https://aprimerahora.pe/2020/06/26/autoridades-anuncian-que-hoy-se-firmando-el-contrato-para-el-hospital-antonio-lorena/'

# Referencias
https://adonde.com/noticias-peru/diarios-lima.php